In [2]:
spark

In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

### 1. Criar um dataframe para ler o arquivo no HDFS /user/<nome/data/juros_selic/juros_selic

In [4]:
# listando o arquivo no HDFS

!hdfs dfs -ls /user/feliciani/data/juros_selic

Found 3 items
-rw-r--r--   3 root supergroup       8348 2021-06-23 18:23 /user/feliciani/data/juros_selic/juros_selic
-rw-r--r--   3 root supergroup      14621 2021-06-23 18:23 /user/feliciani/data/juros_selic/juros_selic.json
-rw-r--r--   3 root supergroup      13147 2021-06-23 18:23 /user/feliciani/data/juros_selic/juros_selic.wsdl


In [5]:
# Conteúdo do Arquivo juros_selic

!hdfs dfs -cat /user/feliciani/data/juros_selic/juros_selic

# tem os campos data e valor, ambos com aspas e separados com ponto e vírgula

In [21]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [7]:
juros_selic = spark.read.csv("/user/feliciani/data/juros_selic/juros_selic", sep=";", header="true")

# informei que o separador é ponto e vírgula e haverá cabeçalho

In [8]:
juros_selic.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
+----------+-----+
only showing top 5 rows



### 2. Agrupar todas as datas pelo ano em ordem decrescente e salvar a quantidade de meses ocorridos, o valor médio, mínimo e máximo do campo valor

In [9]:
juros_selic.show(5)

+----------+-----+
|      data|valor|
+----------+-----+
|01/06/1986| 1,27|
|01/07/1986| 1,95|
|01/08/1986| 2,57|
|01/09/1986| 2,94|
|01/10/1986| 1,96|
+----------+-----+
only showing top 5 rows



In [10]:
# usado o split para separar o campo data, delimitando pela / 

juros_ano = juros_selic.withColumn("ano", split(col("data"), "/")).show(3)

+----------+-----+--------------+
|      data|valor|           ano|
+----------+-----+--------------+
|01/06/1986| 1,27|[01, 06, 1986]|
|01/07/1986| 1,95|[01, 07, 1986]|
|01/08/1986| 2,57|[01, 08, 1986]|
+----------+-----+--------------+
only showing top 3 rows



In [12]:
# usar o getItem para pegar a segunda posição.

juros_ano = juros_selic.withColumn("ano", split(col("data"), "/").getItem(2))

# criado o campo ano

In [13]:
juros_ano.show(3)

+----------+-----+----+
|      data|valor| ano|
+----------+-----+----+
|01/06/1986| 1,27|1986|
|01/07/1986| 1,95|1986|
|01/08/1986| 2,57|1986|
+----------+-----+----+
only showing top 3 rows



In [17]:
# usar o REGEX.  col("valor","\,","\.") sempre que achar uma vírgula substituir por um ponto na coluna valor

juros_valor = juros_ano.withColumn("valor", regexp_replace(col("valor"),"\,","\."))


In [18]:
juros_valor.show(5)

# susbstituiu a vírgula pelo ponto na coluna valor

+----------+-----+----+
|      data|valor| ano|
+----------+-----+----+
|01/06/1986| 1.27|1986|
|01/07/1986| 1.95|1986|
|01/08/1986| 2.57|1986|
|01/09/1986| 2.94|1986|
|01/10/1986| 1.96|1986|
+----------+-----+----+
only showing top 5 rows



In [20]:
# valor está como string

juros_valor.printSchema()

root
 |-- data: string (nullable = true)
 |-- valor: string (nullable = true)
 |-- ano: string (nullable = true)



In [23]:
# cast fará a modificação do tipo 
juros_valor = juros_ano.withColumn("valor", regexp_replace(col("valor"),"\,","\.").cast(FloatType()))

In [24]:
juros_valor.show(5)
juros_valor.printSchema()

# valor está como float e possibilita operações matemáticas

+----------+-----+----+
|      data|valor| ano|
+----------+-----+----+
|01/06/1986| 1.27|1986|
|01/07/1986| 1.95|1986|
|01/08/1986| 2.57|1986|
|01/09/1986| 2.94|1986|
|01/10/1986| 1.96|1986|
+----------+-----+----+
only showing top 5 rows

root
 |-- data: string (nullable = true)
 |-- valor: float (nullable = true)
 |-- ano: string (nullable = true)



In [26]:
# groupby

juros_relatorio = juros_valor.groupby("ano").count().show()

+----+-----+
| ano|count|
+----+-----+
|1987|   12|
|2016|   12|
|2012|   12|
|1988|   12|
|2019|    2|
|2017|   12|
|2014|   12|
|2013|   12|
|2005|   12|
|2000|   12|
|2002|   12|
|2009|   12|
|2018|   12|
|1995|   12|
|2006|   12|
|2004|   12|
|1989|   12|
|2011|   12|
|1992|   12|
|2008|   12|
+----+-----+
only showing top 20 rows



In [31]:
# agregações agg contando os anos
juros_relatorio = juros_valor.groupby("ano").agg(count("ano").alias("meses")).show()

+----+-----+
| ano|meses|
+----+-----+
|1987|   12|
|2016|   12|
|2012|   12|
|1988|   12|
|2019|    2|
|2017|   12|
|2014|   12|
|2013|   12|
|2005|   12|
|2000|   12|
|2002|   12|
|2009|   12|
|2018|   12|
|1995|   12|
|2006|   12|
|2004|   12|
|1989|   12|
|2011|   12|
|1992|   12|
|2008|   12|
+----+-----+
only showing top 20 rows



In [32]:
# agregações agg médias
juros_relatorio = juros_valor.groupby("ano").agg(count("ano").alias("meses"), avg("valor").alias("Valor Médio")).show()

+----+-----+-------------------+
| ano|meses|        Valor Médio|
+----+-----+-------------------+
|1987|   12| 13.520833333333334|
|2016|   12|  1.099999984105428|
|2012|   12| 0.6808333347241083|
|1988|   12| 22.733333428700764|
|2019|    2|0.45500001311302185|
|2017|   12| 0.7941666692495346|
|2014|   12| 0.8666666646798452|
|2013|   12| 0.6600000113248825|
|2005|   12| 1.4633333285649617|
|2000|   12| 1.3491666714350383|
|2002|   12| 1.4716666638851166|
|2009|   12| 0.7916666666666666|
|2018|   12| 0.5199999958276749|
|1995|   12| 3.6133333444595337|
|2006|   12| 1.1774999797344208|
|2004|   12| 1.2616666654745738|
|1989|   12| 31.675833861033123|
|2011|   12| 0.9200000067551931|
|1992|   12| 26.324999968210857|
|2008|   12| 0.9849999944368998|
+----+-----+-------------------+
only showing top 20 rows



In [37]:
# formatando a média para 2 casas após a vírgula

juros_relatorio = juros_valor.groupby("ano").agg(count("ano").alias("meses"), format_number(avg("valor"),2).alias("Valor Médio")).show()

+----+-----+-----------+
| ano|meses|Valor Médio|
+----+-----+-----------+
|1987|   12|      13.52|
|2016|   12|       1.10|
|2012|   12|       0.68|
|1988|   12|      22.73|
|2019|    2|       0.46|
|2017|   12|       0.79|
|2014|   12|       0.87|
|2013|   12|       0.66|
|2005|   12|       1.46|
|2000|   12|       1.35|
|2002|   12|       1.47|
|2009|   12|       0.79|
|2018|   12|       0.52|
|1995|   12|       3.61|
|2006|   12|       1.18|
|2004|   12|       1.26|
|1989|   12|      31.68|
|2011|   12|       0.92|
|1992|   12|      26.32|
|2008|   12|       0.98|
+----+-----+-----------+
only showing top 20 rows



In [38]:
# Valor mínimo

juros_relatorio = juros_valor.groupby("ano").agg(count("ano").alias("meses"), format_number(avg("valor"),2).alias("Valor Médio"), min("valor").alias("Valor Mínimo")).show()

+----+-----+-----------+------------+
| ano|meses|Valor Médio|Valor Mínimo|
+----+-----+-----------+------------+
|1987|   12|      13.52|        7.99|
|2016|   12|       1.10|         1.0|
|2012|   12|       0.68|        0.54|
|1988|   12|      22.73|       16.59|
|2019|    2|       0.46|        0.37|
|2017|   12|       0.79|        0.54|
|2014|   12|       0.87|        0.77|
|2013|   12|       0.66|        0.49|
|2005|   12|       1.46|        1.22|
|2000|   12|       1.35|         1.2|
|2002|   12|       1.47|        1.25|
|2009|   12|       0.79|        0.66|
|2018|   12|       0.52|        0.47|
|1995|   12|       3.61|        2.78|
|2006|   12|       1.18|        0.99|
|2004|   12|       1.26|        1.08|
|1989|   12|      31.68|       11.43|
|2011|   12|       0.92|        0.84|
|1992|   12|      26.32|        23.0|
|2008|   12|       0.98|         0.8|
+----+-----+-----------+------------+
only showing top 20 rows



In [39]:

juros_relatorio = juros_valor.groupby("ano").agg(count("ano").alias("meses"), format_number(avg("valor"),2).alias("Valor Médio"), min("valor").alias("Valor Mínimo"), max("valor").alias("Valor Máximo")).show()

+----+-----+-----------+------------+------------+
| ano|meses|Valor Médio|Valor Mínimo|Valor Máximo|
+----+-----+-----------+------------+------------+
|1987|   12|      13.52|        7.99|       24.63|
|2016|   12|       1.10|         1.0|        1.22|
|2012|   12|       0.68|        0.54|        0.89|
|1988|   12|      22.73|       16.59|       30.24|
|2019|    2|       0.46|        0.37|        0.54|
|2017|   12|       0.79|        0.54|        1.09|
|2014|   12|       0.87|        0.77|        0.96|
|2013|   12|       0.66|        0.49|        0.81|
|2005|   12|       1.46|        1.22|        1.66|
|2000|   12|       1.35|         1.2|        1.49|
|2002|   12|       1.47|        1.25|        1.74|
|2009|   12|       0.79|        0.66|        1.05|
|2018|   12|       0.52|        0.47|        0.58|
|1995|   12|       3.61|        2.78|        4.26|
|2006|   12|       1.18|        0.99|        1.43|
|2004|   12|       1.26|        1.08|        1.48|
|1989|   12|      31.68|       

In [43]:
# ordenando os anos de forma decrescente

juros_relatorio = juros_valor.groupby("ano").agg(count("ano").alias("meses"), format_number(avg("valor"),2).alias("Valor Médio"), min("valor").alias("Valor Mínimo"), max("valor").alias("Valor Máximo")).sort(desc("ano"))

In [44]:
juros_relatorio.show()

+----+-----+-----------+------------+------------+
| ano|meses|Valor Médio|Valor Mínimo|Valor Máximo|
+----+-----+-----------+------------+------------+
|2019|    2|       0.46|        0.37|        0.54|
|2018|   12|       0.52|        0.47|        0.58|
|2017|   12|       0.79|        0.54|        1.09|
|2016|   12|       1.10|         1.0|        1.22|
|2015|   12|       1.04|        0.82|        1.18|
|2014|   12|       0.87|        0.77|        0.96|
|2013|   12|       0.66|        0.49|        0.81|
|2012|   12|       0.68|        0.54|        0.89|
|2011|   12|       0.92|        0.84|        1.07|
|2010|   12|       0.78|        0.59|        0.93|
|2009|   12|       0.79|        0.66|        1.05|
|2008|   12|       0.98|         0.8|        1.18|
|2007|   12|       0.94|         0.8|        1.08|
|2006|   12|       1.18|        0.99|        1.43|
|2005|   12|       1.46|        1.22|        1.66|
|2004|   12|       1.26|        1.08|        1.48|
|2003|   12|       1.76|       

### 3. Salvar no hdfs:///user/<nome>/relatorioAnual com compressão zlib e formato orc

In [45]:
juros_relatorio.write.orc("/user/feliciani/relatorio_anual", compression="zlib")

In [46]:
!hdfs dfs -ls /user/feliciani/relatorio_anual

# Salvou em várias partições com formato ORC e compressão zlib

Found 35 items
-rw-r--r--   2 root supergroup          0 2021-06-29 21:22 /user/feliciani/relatorio_anual/_SUCCESS
-rw-r--r--   2 root supergroup        582 2021-06-29 21:21 /user/feliciani/relatorio_anual/part-00000-797dd3b3-3809-427c-9f74-5be35decb4d0-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2021-06-29 21:21 /user/feliciani/relatorio_anual/part-00001-797dd3b3-3809-427c-9f74-5be35decb4d0-c000.zlib.orc
-rw-r--r--   2 root supergroup        585 2021-06-29 21:21 /user/feliciani/relatorio_anual/part-00002-797dd3b3-3809-427c-9f74-5be35decb4d0-c000.zlib.orc
-rw-r--r--   2 root supergroup        572 2021-06-29 21:21 /user/feliciani/relatorio_anual/part-00003-797dd3b3-3809-427c-9f74-5be35decb4d0-c000.zlib.orc
-rw-r--r--   2 root supergroup        583 2021-06-29 21:21 /user/feliciani/relatorio_anual/part-00004-797dd3b3-3809-427c-9f74-5be35decb4d0-c000.zlib.orc
-rw-r--r--   2 root supergroup        582 2021-06-29 21:21 /user/feliciani/relatorio_anual/part-00005-797dd3b3-3809-427c